# soru:

- doktora başvurularına ne zaman bakmalıyım?

- discarded featureleri biz de deneyip sonucu etkilemediğini görsek mi?

# todo:

- [ ] df.name kullanımları df["name"]'ye çevrilecek

- [ ] match type kolonu 3 kolona ayrılacak, m8, seed_match & a1

- [ ] TargetScan miRNA databasesindeki conservation değerleri yeni bir kolon olarak (mirna_conservation) eklenecek

- [ ] TargetScan'den her seed için ve her 2 çeşit match için (6mer, 7mer) SPS değerleri alınıp kolon olarak eklenecek

- [ ] target_abundance kolonu eklenecek (hedef UTR'da o miRNA'nın kaç tane non-overlapping hedefi var?)

- [ ] figure out a way to implement target_conservation (TargetScan uses Pct)

- [x] extended_3utr_supplementary_pairing kolonu (12-17 nuc) eklenebilir
    - option olarak eklendi

- [ ] sekans alan kısımlar get_dna_seq_from_coordinates ile değiştirilmeli

## added

- flanking dinucleotides
    - match type agnostic yaptım, makaleden öyle anladığım için
    - sekansları yazmak yerine score ekledim (deep learning yaparsak raw sequence'ye çevrilmeli)

In [1]:
from utils import *
import pandas as pd

from icecream import ic

### importing sequence & miRNA dataframe

In [2]:
# parameters for TP53

seq_file = "../sequences/tp53/Homo_sapiens_ENST00000420246_2_sequence.fa"
canonical_result_file = (
    "../sequences/tp53/TargetScan_8.0_ENST00000420246.2_predicted_targeting_details.txt"
)

In [3]:
sequence = import_fasta(seq_file)

targetscan_df = pd.read_csv("../data/mirna_databases/targetscan.tsv", sep="\t")

targetscan_df.head()

,name,sequence,seed,conservation,accession
0,hsa-let-7a-5p,AACUAUACAACCUACUACCUCA,UACCUC,2,MIMAT0000062
1,hsa-let-7b-5p,AACCACACAACCUACUACCUCA,UACCUC,2,MIMAT0000063
2,hsa-let-7c-5p,AACCAUACAACCUACUACCUCA,UACCUC,2,MIMAT0000064
3,hsa-let-7d-5p,AACUAUGCAACCUACUACCUCU,UACCUC,2,MIMAT0000065
4,hsa-let-7e-5p,AACUAUACAACCUCCUACCUCA,UACCUC,2,MIMAT0000066


### results

In [4]:
results_df = find_matches(sequence, targetscan_df,
                          ignore_first_15_nucleotides=True)

results_df.head()

,name,start,end,seed match,match_type,mirna_sequence
2,hsa-miR-376a-3p,19,25,UCUAUGA,7mer-m8,ACGUGGAUUUUCCUCUAUGAU
3,hsa-miR-376b-3p,19,25,UCUAUGA,7mer-m8,AACAUGGAUUUUCCUCUAUGAU
4,hsa-miR-4269,29,35,UGCCUGA,7mer-A1,GCCAGGGCUGUCUGUGCCUGC
5,hsa-miR-6715b-5p,29,35,UGCCUGA,7mer-A1,UGCCAAACCAGUCGUGCCUGU
6,hsa-miR-4514,29,35,UGCCUGA,7mer-A1,UUCCCCAAUCCUGCCUGU


In [5]:
print_results(results_df, sequence, summary=True)

,results,match_types
0,5' AGAGAGCATGAAAATGGTTCTATGA 3' position: 1-25 of DNA | || | | ||||||| 3' UGCACCUAAAAGGAGAUACUA 5' hsa-miR-376a-3p,7mer-m8
1,5' AGAGAGCATGAAAATGGTTCTATGA 3' position: 1-25 of DNA | |||| | | ||||||| 3' UUGUACCUAAAAGGAGAUACUA 5' hsa-miR-376b-3p,7mer-m8
2,5' GCATGAAAATGGTTCTATGACTTTGCCTGA 3' position: 6-35 of DNA || || |||||| 3' CGGUCCCGACAGACACGGACG 5' hsa-miR-4269,7mer-A1
3,5' GCATGAAAATGGTTCTATGACTTTGCCTGA 3' position: 6-35 of DNA || |||||| 3' ACGGUUUGGUCAGCACGGACA 5' hsa-miR-6715b-5p,7mer-A1
4,5' GCATGAAAATGGTTCTATGACTTTGCCTGA 3' position: 6-35 of DNA ||| | |||||| 3' AAGGGGUUAGGACGGACA 5' hsa-miR-4514,7mer-A1


In [6]:
generate_3utr_length_column(sequence, results_df)
generate_3_supplementary_pairing_column(sequence, results_df)


results_df[results_df["3_supplementary_pairing"] == 1]

,name,start,end,seed match,match_type,mirna_sequence,3utr_length,3_supplementary_pairing
571,hsa-miR-6884-5p,1302,1308,CAGCCUC,7mer-m8,CAACAUCACCUUCUCAGCCUCU,1494,1
584,hsa-miR-6780a-5p,1305,1311,CCUCCCA,7mer-m8,UCUCCAGCUGUCUUCCCUCCCAA,1494,1
628,hsa-miR-4271,1405,1411,CUUCCCC,7mer-m8,CCCCACCUUUUCUUCCCCC,1494,1
638,hsa-miR-5703,1414,1420,CUUCUCC,7mer-m8,ACCUUCCCGACUUCUCCU,1494,1


In [7]:
generate_3_supplementary_pairing_column(sequence, results_df, extended=True)

results_df[results_df["extended_supplementary_pairing"] == 1]

,name,start,end,seed match,match_type,mirna_sequence,3utr_length,3_supplementary_pairing,extended_supplementary_pairing
638,hsa-miR-5703,1414,1420,CUUCUCC,7mer-m8,ACCUUCCCGACUUCUCCU,1494,1,1


In [8]:
generate_avg_position_column(results_df)
generate_close_proximity_column(results_df)

results_df[results_df["close_proximity"] == 1]

,name,start,end,seed match,match_type,mirna_sequence,3utr_length,3_supplementary_pairing,extended_supplementary_pairing,avg_position,close_proximity
124,hsa-miR-223-3p,285,291,ACUGACA,7mer-A1,UGGGGUAUUUGACAAACUGACA,1494,0,0,288,1
125,hsa-miR-4697-3p,285,291,ACUGACA,7mer-A1,ACCAAGGGGCAGGAGUCACUGACA,1494,0,0,288,1
140,hsa-miR-223-3p,312,318,ACUGACA,7mer-A1,UGGGGUAUUUGACAAACUGACA,1494,0,0,315,1
141,hsa-miR-4697-3p,311,318,CACUGACA,8mer,ACCAAGGGGCAGGAGUCACUGACA,1494,0,0,314,1
536,hsa-miR-4786-5p,1242,1248,GGUCUCA,7mer-A1,GGUGCAUCCAGUCCUGGUCUCA,1494,0,0,1245,1
537,hsa-miR-769-5p,1242,1248,GGUCUCA,7mer-A1,AGCUCAGAACCCAGAGGUCUCA,1494,0,0,1245,1
550,hsa-miR-4786-5p,1264,1271,UGGUCUCA,8mer,GGUGCAUCCAGUCCUGGUCUCA,1494,0,0,1267,1
551,hsa-miR-769-5p,1265,1271,GGUCUCA,7mer-A1,AGCUCAGAACCCAGAGGUCUCA,1494,0,0,1268,1


In [9]:
generate_total_no_of_pairs_column(sequence, results_df)

results_df.sort_values("total_no_of_pairs", ascending=False).head()

,name,start,end,seed match,match_type,mirna_sequence,3utr_length,3_supplementary_pairing,extended_supplementary_pairing,avg_position,close_proximity,total_no_of_pairs
477,hsa-miR-1285-3p,1092,1098,UGCCCAG,7mer-m8,AGGUCUCACUUUGUUGCCCAGA,1494,0,0,1095,0,19
542,hsa-miR-1285-3p,1255,1261,UGCCCAG,7mer-m8,AGGUCUCACUUUGUUGCCCAGA,1494,0,0,1258,0,18
441,hsa-miR-4685-5p,963,969,CCCUGGA,7mer-A1,AACCUUGCCCCACUCCAAGCCCUGGG,1494,0,0,966,0,18
168,hsa-miR-6803-5p,326,332,ACCCCCA,7mer-m8,ACGCCCAGCCCCCCACCCCCAG,1494,0,0,329,0,17
78,hsa-miR-7850-5p,217,224,GUCCAAAA,8mer,CCAGCCACACUAUGUCCAAAC,1494,0,0,220,0,16


In [10]:
generate_position_in_utr_column(sequence, results_df).head()

,name,start,end,seed match,match_type,mirna_sequence,3utr_length,3_supplementary_pairing,extended_supplementary_pairing,avg_position,close_proximity,total_no_of_pairs,position_in_utr
2,hsa-miR-376a-3p,19,25,UCUAUGA,7mer-m8,ACGUGGAUUUUCCUCUAUGAU,1494,0,0,22,0,12,0.015
3,hsa-miR-376b-3p,19,25,UCUAUGA,7mer-m8,AACAUGGAUUUUCCUCUAUGAU,1494,0,0,22,0,14,0.015
4,hsa-miR-4269,29,35,UGCCUGA,7mer-A1,GCCAGGGCUGUCUGUGCCUGC,1494,0,0,32,0,10,0.021
5,hsa-miR-6715b-5p,29,35,UGCCUGA,7mer-A1,UGCCAAACCAGUCGUGCCUGU,1494,0,0,32,0,8,0.021
6,hsa-miR-4514,29,35,UGCCUGA,7mer-A1,UUCCCCAAUCCUGCCUGU,1494,0,0,32,0,10,0.021


In [11]:
generate_flanking_scores_column(sequence, results_df)

results_df.head()

,name,start,end,seed match,match_type,mirna_sequence,3utr_length,3_supplementary_pairing,extended_supplementary_pairing,avg_position,close_proximity,total_no_of_pairs,position_in_utr,5end_flank_score,3end_flank_score
2,hsa-miR-376a-3p,19,25,UCUAUGA,7mer-m8,ACGUGGAUUUUCCUCUAUGAU,1494,0,0,22,0,12,0.015,-2,1
3,hsa-miR-376b-3p,19,25,UCUAUGA,7mer-m8,AACAUGGAUUUUCCUCUAUGAU,1494,0,0,22,0,14,0.015,-2,1
4,hsa-miR-4269,29,35,UGCCUGA,7mer-A1,GCCAGGGCUGUCUGUGCCUGC,1494,0,0,32,0,10,0.021,1,2
5,hsa-miR-6715b-5p,29,35,UGCCUGA,7mer-A1,UGCCAAACCAGUCGUGCCUGU,1494,0,0,32,0,8,0.021,1,2
6,hsa-miR-4514,29,35,UGCCUGA,7mer-A1,UUCCCCAAUCCUGCCUGU,1494,0,0,32,0,10,0.021,1,2


In [14]:
def generate_local_au_content_column(sequence, results_df):
    
    names = results_df["name"].tolist()
    starts = results_df["start"].tolist()
    ends = results_df["end"].tolist()

    # generating weights that correspond to each position in the 30nt window
    weight_dict_5end = {i:(1/(32-i)) for i in range(1,31)}
    weight_dict_3end = {i:(1/(i+1)) for i in range(1,31)}


    for i, _ in enumerate(names):
        
        start_5end = (starts[i]-30)if starts[i]>30 else 1
        end_5end = starts[i]-1

        start_3end = ends[i]+1
        end_3end = ends[i]+31

        # getting flanking 30nt sequences
        window_5end = get_dna_seq_from_coordinates(sequence, start_5end, end_5end)
        window_3end = get_dna_seq_from_coordinates(sequence, start_3end, end_3end)

        score_5end = sum(
            weight_dict_5end[i + 1]
            for i, nucleotide in enumerate(window_5end)
            if nucleotide in ["A", "U"]
        )
        
        score_3end = sum(
            weight_dict_3end[i + 1]
            for i, nucleotide in enumerate(window_3end)
            if nucleotide in ["A", "U"]
        )

generate_local_au_content_column(sequence, results_df)    